In [8]:
import json
import pandas as pd
import geopandas as gpd
import bokeh
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import all_palettes
from bokeh.models import HoverTool
from bokeh.layouts import row, gridplot, column
from bokeh.models import Div

Load the brazil shapefile by uf: 

In [9]:
update_columns_uf = {'ACRE':'AC', 'ALAGOAS':'AL', 'AMAPA':'AP', 'AMAZONAS':'AM', 'BAHIA':'BA', 
                     'CEARA':'CE', 'DISTRITO FEDERAL':'DF', 'ESPIRITO SANTO':'ES', 'GOIAS':'GO',
                     'MARANHAO':'MA', 'MATO GROSSO':'MT', 'MATO GROSSO DO SUL':'MS', 'MINAS GERAIS':'MG',
                     'PARA':'PA', 'PARAIBA':'PB', 'PARANA':'PR', 'PERNAMBUCO':'PE', 'PIAUI':'PI', 'RIO DE JANEIRO':'RJ',
                   'RIO GRANDE DO NORTE':'RN', 'RIO GRANDE DO SUL':'RS', 'RONDONIA':'RO', 'RORAIMA':'RR',
                    'SANTA CATARINA':'SC', 'SAO PAULO':'SP', 'SERGIPE':'SE', 'TOCANTINS':'TO'}

In [10]:
gdf = gpd.read_file('./data/shapefile_BR/states.shp')
# selection the useful columns 
gdf = gdf[['ADM1_NAME', 'geometry']]
gdf['ADM1_NAME'] = gdf['ADM1_NAME'].replace(update_columns_uf)
gdf = gdf.rename(columns = {'ADM1_NAME': 'UF'})
gdf.head()

/opt/homebrew/Caskroom/miniforge/base/envs/alerta_dengue/lib/python3.9/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,UF,geometry
0,AC,"POLYGON ((-66.80970 -9.81766, -66.62442 -9.898..."
1,AL,"POLYGON ((-35.14920 -8.91435, -35.16193 -8.950..."
2,AP,"MULTIPOLYGON (((-50.00722 0.88168, -50.04045 0..."
3,AM,"POLYGON ((-58.89492 0.26385, -58.89449 -0.0106..."
4,BA,"MULTIPOLYGON (((-38.91168 -13.67529, -38.94611..."


Load the data with incidence:

In [11]:
agravo = 'dengue'
df = pd.read_csv(f'./data/{agravo}_br_2010-2022_quarter.csv')
df.set_index('data_iniSE', inplace = True)
df.head()

,casos,pop,inc,trimestre,UF,trimestre-tick,regiao
data_iniSE,,,,,,,
2010-03-31,9888,15989929.0,0.000618,201001,RJ,1° - 2010,Sudeste
2010-06-30,20522,15989929.0,0.001283,201002,RJ,2° - 2010,Sudeste
2010-09-30,7405,15989929.0,0.000463,201003,RJ,3° - 2010,Sudeste
2010-12-31,5628,15989929.0,0.000352,201004,RJ,4° - 2010,Sudeste
2011-03-31,73770,16112637.0,0.004578,201101,RJ,1° - 2011,Sudeste


Selecting a specific date and merging the datasets:

In [12]:
df_s = df.loc[df.index == '2022-03-31']

merge = df_s.reset_index().merge(gdf, left_on = 'UF', right_on = 'UF', how = 'left')

# selection columns that it will be used:

merge = merge[['UF', 'inc', 'geometry']]

merge = gpd.GeoDataFrame(data=merge, geometry='geometry')

merge.head()

,UF,inc,geometry
0,RJ,0.000189,"MULTIPOLYGON (((-44.08881 -23.17438, -44.10173..."
1,ES,0.000905,"MULTIPOLYGON (((-29.84003 -20.49640, -29.84888..."
2,PR,0.005675,"MULTIPOLYGON (((-48.28241 -25.48608, -48.28778..."
3,CE,0.002139,"POLYGON ((-37.24091 -4.83142, -37.25617 -4.837..."
4,MA,0.000356,"MULTIPOLYGON (((-44.61029 -3.00800, -44.62000 ..."


In bokeh it's necessary to load this data as json: 

In [13]:
#Read data to json
merged_json = json.loads(merge.to_json())

#Convert to str like object
json_data = json.dumps(merged_json)
    
    
geosource = GeoJSONDataSource(geojson = json_data)


Making the map plot: 

In [14]:
palette = all_palettes['Viridis'][7]
    #palette = palette[::-1]


hover = HoverTool(tooltips = [ ('UF','@UF'),('Incidence', '@inc')])



color_mapper = LinearColorMapper(palette = palette, low = merge['inc'].min(), 
                                high = merge['inc'].max())

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=5, width = 15, height = 300,
border_line_color=None, orientation = 'vertical', location = (0,-1))

p = figure(title = 'Incidence in the 1° quarter of 2022', height = 500 ,width = 600, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs','ys', source = geosource, fill_color = {'field' :'inc', 'transform' : color_mapper},
              line_color = 'black', line_width = 0.25, fill_alpha = 1)


p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
p.yaxis.major_label_text_font_size = '0pt'

p.add_layout(color_bar, 'right')
output_notebook()
show(p)

Loading BokehJS ...